In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [3]:
# Given a m x n matrix  of m closing prices for n equities, returns an 1 x n 
# array of average closing prices
def compute_average(closing_prices_for_multiple_equities):
  sums = [0] * len(closing_prices_for_multiple_equities[0])
  for row in closing_prices_for_multiple_equities:
    for i in range(len(row)):
      sums[i] += row[i]

  avgs = [x/len(closing_prices_for_multiple_equities) for x in sums]
  return avgs

In [4]:
# Given an m x n matrix of m closing prices for n equities and n averages for n 
# equities, returns all of the difference of the closing prices and their averages
# essentially substracts mean to make the new mean of the data set 0 for each column
def compute_stocks_demeaned(closing_prices_matrix, average_closing_prices):
  m = len(closing_prices_matrix)
  n = len(closing_prices_matrix[0])
  demeaned = [[0 for i in range(n)] for j in range(m)]
  for i in range(len(closing_prices_matrix)):
    for j in range(len(closing_prices_matrix[i])):
      demeaned[i][j] = closing_prices_matrix[i][j] - average_closing_prices[j]

  return demeaned

In [5]:
# Given an m x n matrix of the demeaned closing prices for n equities, returns
# the associated covariance matrix calculated by demeaned transpose x demeaned.
def compute_covariance_matrix(demeaned):
    s_minus_m = np.array(demeaned)
    s_minus_m_t = demeaned.transpose()
    return np.matmul(s_minus_m_t, s_minus_m)/len(demeaned)

In [6]:
def estimated_portfolio_risk_based_on_stddev (asset_weights, covariance_matrix):
  Wt = np.asmatrix(asset_weights)
  W = Wt.transpose()
  return np.sqrt(Wt.dot(covariance_matrix).dot(W))

In [7]:
def expected_portfolio_return(average_closing_prices, asset_weights):
  M = np.array(average_closing_prices)
  W = np.array(asset_weights)
  return M.dot(W)

In [8]:
def sharpe_ratio(expected_return, portfolio_std_dev, risk_free_rate):
  return (expected_return - risk_free_rate)/portfolio_std_dev

In [9]:
# Given array of equities, returning dataframe with historical data dating back 
def get_closing_price_historical_data(equities, period, interval):
  data = {}
  for x in equities:
    ticker = yf.Ticker(x)
    history = ticker.history(period= period, interval= interval)['Close']
    data[x] = history
  return data

In [10]:
senbet = pd.read_excel('Senbet Portfolio.xlsx')
senbet

,$,Allocation,Shares
0,MCD,0.0211,142
1,AMZN.O,0.0246,12
2,FIVE.O,0.0213,169
3,TJX,0.0197,451
4,DAN,0.0266,1650
5,PG,0.0212,236
6,STZ,0.0146,97
7,EL,0.0204,106
8,COST.O,0.0201,86
9,ZTS,0.0270,259


In [11]:
trimmed_senbet = senbet
trimmed_senbet.columns = ['Ticker', 'Allocation', 'Shares']
trimmed_senbet['Allocation'] *= 100
print(trimmed_senbet)

                Ticker  Allocation  Shares
0                  MCD        2.11     142
1               AMZN.O        2.46      12
2               FIVE.O        2.13     169
3                  TJX        1.97     451
4   DAN                       2.66    1650
5                   PG        2.12     236
6                  STZ        1.46      97
7                   EL        2.04     106
8               COST.O        2.01      86
9                  ZTS        2.70     259
10                 SYK        2.42     150
11              ICLR.O        2.25     173
12                 BMY        2.48     593
13                 DGX        2.38     280
14              ABBV.K        2.82     394
15                 HON        2.96     206
16                 RTX        2.37     464
17                  WM        2.64     309
18              AAPL.O        2.28     282
19              MSFT.O        2.68     172
20                   V        3.31     236
21              CSCO.O        2.46     718
22         

In [12]:
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.O', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace('.K', ''), axis = 1)
trimmed_senbet['Ticker'] = trimmed_senbet.apply(lambda x: x['Ticker'].replace(' ', ''), axis = 1)
trimmed_senbet

,Ticker,Allocation,Shares
0,MCD,2.11,142
1,AMZN,2.46,12
2,FIVE,2.13,169
3,TJX,1.97,451
4,DAN,2.66,1650
5,PG,2.12,236
6,STZ,1.46,97
7,EL,2.04,106
8,COST,2.01,86
9,ZTS,2.70,259


In [13]:
asset_weights = list(trimmed_senbet['Allocation']/100)
asset_weights

[0.021099999999999997,
 0.0246,
 0.0213,
 0.0197,
 0.0266,
 0.0212,
 0.0146,
 0.0204,
 0.020099999999999996,
 0.027000000000000003,
 0.0242,
 0.0225,
 0.0248,
 0.0238,
 0.0282,
 0.0296,
 0.023700000000000002,
 0.0264,
 0.0228,
 0.0268,
 0.0331,
 0.0246,
 0.0232,
 0.0284,
 0.0259,
 0.025,
 0.0202,
 0.0208,
 0.0109,
 0.0138,
 0.03,
 0.0308,
 0.0239,
 0.03,
 0.0138,
 0.0136,
 0.0162,
 0.0098,
 0.015599999999999998,
 0.012199999999999999,
 0.022,
 0.0216,
 0.022099999999999998,
 0.022399999999999996,
 0.0222]

In [14]:
historical_data = get_closing_price_historical_data(trimmed_senbet['Ticker'], '179d', '1d')

In [15]:
portfolio = pd.DataFrame(historical_data)
# historical_df =historical_df.dropna(axis = 0)
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-17,227.887772,3270.389893,183.710007,71.480980,27.570318,135.295410,234.241241,295.134857,382.622589,169.851974,...,70.882240,707.123535,169.917068,155.147995,25.716753,111.560532,364.305054,155.673111,68.210098,48.401932
2021-05-18,228.133682,3232.280029,184.710007,70.617371,26.657919,133.910339,232.493408,294.697113,381.476593,169.692352,...,71.266731,710.271118,170.683838,152.585159,25.228025,109.864349,359.677887,153.817261,66.992050,48.650143
2021-05-19,226.382828,3231.800049,181.389999,66.884987,26.102545,134.185394,230.775360,296.915771,378.337585,170.859497,...,71.631493,710.300842,170.143173,147.498016,24.661102,109.844513,353.580658,153.559204,66.304459,48.481358
2021-05-20,228.379593,3247.679932,182.449997,66.875069,25.824860,135.560638,234.469666,296.458130,382.243927,175.368393,...,73.238411,722.970642,172.168213,145.147171,24.817495,111.054642,354.961884,154.353165,66.078529,48.759361
2021-05-21,227.455002,3203.080078,179.070007,66.547493,26.499243,135.560638,235.641510,295.244293,379.393890,175.587860,...,73.386299,720.091125,171.637360,145.214600,24.964115,112.244949,361.355164,155.683014,67.050995,48.540928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,253.610001,2890.879883,169.740005,69.059998,22.719999,161.300003,240.059998,301.790009,488.899994,200.070007,...,81.919998,730.840027,195.259995,206.000000,27.070000,125.320000,343.390015,158.929993,68.610001,53.619999
2022-01-25,250.679993,2799.719971,162.600006,68.680000,22.320000,159.539993,236.470001,294.209991,477.320007,195.250000,...,75.099998,719.669983,194.160004,213.139999,27.930000,123.949997,341.549988,173.110001,68.989998,53.849998
2022-01-26,249.850006,2777.449951,155.660004,69.099998,22.219999,158.149994,234.330002,292.570007,483.470001,189.839996,...,72.639999,695.830017,189.720001,212.449997,27.520000,122.150002,342.679993,175.320007,69.150002,52.730000


In [16]:
for index, x in trimmed_senbet.iterrows():
    portfolio[x.Ticker]*=x.Shares

In [17]:
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,NEE,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-17,32360.063568,39244.678711,31046.991135,32237.921944,45491.025066,31929.716797,22721.400421,31284.294861,32905.542664,43991.661392,...,19279.969360,25456.447266,15292.536163,22961.903259,21936.390314,33133.477890,36430.505371,36738.854187,37310.923752,36446.654617
2021-05-18,32394.982880,38787.360352,31215.991135,31848.434143,43985.566235,31602.840088,22551.860596,31237.893982,32806.987000,43950.319244,...,19384.550903,25569.760254,15361.545410,22582.603577,21519.505697,32629.711761,35967.788696,36300.873535,36644.651443,36633.557430
2021-05-19,32146.361542,38781.600586,30654.909897,30165.129082,43069.198895,31667.753052,22385.209930,31473.071777,32537.032349,44252.609741,...,19483.765991,25570.830322,15312.885590,21829.706421,21035.920258,32623.820343,35358.065796,36239.972168,36268.538864,36506.462254
2021-05-20,32429.902191,38972.159180,30834.049484,30160.655968,42611.018372,31992.310669,22743.557556,31424.561768,32872.977722,45420.413773,...,19920.847778,26026.943115,15495.139160,21481.781311,21169.323530,32983.228592,35496.188354,36427.346863,36144.955559,36715.799034
2021-05-21,32298.610260,38436.960938,30262.831238,30012.919334,43723.750591,31992.310669,22857.226471,31295.895081,32627.874573,45477.255768,...,19961.073364,25923.280518,15447.362366,21491.760742,21294.390217,33336.749954,36135.516357,36741.191284,36676.894196,36551.318699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,36012.620087,34690.558594,28686.060928,31146.058899,37487.998867,38066.800720,23285.819763,31989.740906,42045.399475,51818.131897,...,22282.239502,26310.240967,17573.399506,30488.000000,23090.709740,37220.039909,34339.001465,37507.478271,37529.670334,40375.859196
2022-01-25,35596.558960,33596.639648,27479.401031,30974.680138,36827.999496,37651.438416,22937.590118,31186.259094,41049.520630,50569.750000,...,20427.199585,25908.119385,17474.400330,31544.719910,23824.290260,36813.149094,34154.998779,40853.960144,37737.528831,40549.048851
2022-01-26,35478.700867,33329.399414,26306.540619,31164.099312,36662.998867,37323.398560,22730.010178,31012.420776,41578.420105,49168.559052,...,19758.079834,25049.880615,17074.800110,31442.599548,23474.560390,36278.550453,34267.999268,41375.521729,37825.050835,39705.689655


In [21]:
portfolio["PortValue"] = portfolio.sum(axis=1)
portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM,PortValue
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-17,32360.063568,39244.678711,31046.991135,32237.921944,45491.025066,31929.716797,22721.400421,31284.294861,32905.542664,43991.661392,...,25456.447266,15292.536163,22961.903259,21936.390314,33133.477890,36430.505371,36738.854187,37310.923752,36446.654617,1.567654e+06
2021-05-18,32394.982880,38787.360352,31215.991135,31848.434143,43985.566235,31602.840088,22551.860596,31237.893982,32806.987000,43950.319244,...,25569.760254,15361.545410,22582.603577,21519.505697,32629.711761,35967.788696,36300.873535,36644.651443,36633.557430,1.558539e+06
2021-05-19,32146.361542,38781.600586,30654.909897,30165.129082,43069.198895,31667.753052,22385.209930,31473.071777,32537.032349,44252.609741,...,25570.830322,15312.885590,21829.706421,21035.920258,32623.820343,35358.065796,36239.972168,36268.538864,36506.462254,1.553402e+06
2021-05-20,32429.902191,38972.159180,30834.049484,30160.655968,42611.018372,31992.310669,22743.557556,31424.561768,32872.977722,45420.413773,...,26026.943115,15495.139160,21481.781311,21169.323530,32983.228592,35496.188354,36427.346863,36144.955559,36715.799034,1.572704e+06
2021-05-21,32298.610260,38436.960938,30262.831238,30012.919334,43723.750591,31992.310669,22857.226471,31295.895081,32627.874573,45477.255768,...,25923.280518,15447.362366,21491.760742,21294.390217,33336.749954,36135.516357,36741.191284,36676.894196,36551.318699,1.573361e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,36012.620087,34690.558594,28686.060928,31146.058899,37487.998867,38066.800720,23285.819763,31989.740906,42045.399475,51818.131897,...,26310.240967,17573.399506,30488.000000,23090.709740,37220.039909,34339.001465,37507.478271,37529.670334,40375.859196,1.664029e+06
2022-01-25,35596.558960,33596.639648,27479.401031,30974.680138,36827.999496,37651.438416,22937.590118,31186.259094,41049.520630,50569.750000,...,25908.119385,17474.400330,31544.719910,23824.290260,36813.149094,34154.998779,40853.960144,37737.528831,40549.048851,1.637285e+06
2022-01-26,35478.700867,33329.399414,26306.540619,31164.099312,36662.998867,37323.398560,22730.010178,31012.420776,41578.420105,49168.559052,...,25049.880615,17074.800110,31442.599548,23474.560390,36278.550453,34267.999268,41375.521729,37825.050835,39705.689655,1.628763e+06


In [19]:
import statistics

In [22]:
pct_change = []
for i in range(1, len(portfolio["PortValue"])):
    curr = portfolio["PortValue"][i]
    yester = portfolio["PortValue"][i-1]
    delta = curr-yester
    pct_change.append(delta/curr * 100)

In [23]:
avg_pct_change = statistics.mean(pct_change)
print(avg_pct_change)

-0.4143224924493599


In [24]:
stdev_pct_change = statistics.stdev(pct_change)
print(stdev_pct_change)

11.042289577060739


In [25]:
print((avg_pct_change-.0145)/stdev_pct_change)

-0.03883456319966433


In [26]:
for sym in portfolio.columns:
    data = list(portfolio[sym])
    pct_change = []
    for i in range(1, len(data)):
        curr = data[i]
        yester = data[i-1]
        delta = curr-yester
        pct_change.append(delta/curr * 100)
    pct_change.insert(0, 0)
    portfolio[sym] = pct_change

portfolio

,MCD,AMZN,FIVE,TJX,DAN,PG,STZ,EL,COST,ZTS,...,EQIX,ARE,PXD,BKR,ICE,GS,AXP,KBWB,BAM,PortValue
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-05-18,0.107792,-1.179040,0.541389,-1.222942,-3.422620,-1.034327,-0.751778,-0.148540,-0.300411,-0.094066,...,0.443152,0.449234,-1.679610,-1.937241,-1.543888,-1.286475,-1.206529,-1.818198,0.510196,-0.584846
2021-05-19,-0.773404,-0.014852,-1.830314,-5.580301,-2.127663,0.204981,-0.744468,0.747235,-0.829684,0.683102,...,0.004185,-0.317770,-3.448957,-2.298856,-0.018059,-1.724424,-0.168050,-1.037022,-0.348144,-0.330704
2021-05-20,0.874319,0.488961,0.580980,-0.014831,-1.075263,1.014486,1.575601,-0.154370,1.021950,2.571099,...,1.752464,1.176198,-1.619629,0.630173,1.089670,0.389119,0.514379,-0.341910,0.570155,1.227299
2021-05-21,-0.406494,-1.392405,-1.887524,-0.492243,2.544915,0.000000,0.497300,-0.411130,-0.751208,0.124990,...,-0.399882,-0.309288,0.046434,0.587322,1.060455,1.769251,0.854203,1.450337,-0.449998,0.041752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.386418,1.315163,6.286083,2.316824,2.684854,-0.818346,-0.395733,1.580576,1.491104,-0.129952,...,1.267036,-0.676025,-0.441749,-0.628002,-0.255346,-0.151428,0.119541,0.976531,0.428944,0.684488
2022-01-25,-1.168824,-3.256037,-4.391143,-0.553287,-1.792113,-1.103178,-1.518161,-2.576397,-2.426043,-2.468634,...,-1.552106,-0.566538,3.349911,3.079129,-1.105287,-0.538728,8.191328,0.550801,0.427112,-1.633437
2022-01-26,-0.332194,-0.801815,-4.458437,0.607812,-0.450047,-0.878912,-0.913242,-0.560544,1.272053,-2.849770,...,-3.426119,-2.340292,-0.324783,-1.489825,-1.473594,0.329755,1.260556,0.231386,-2.124026,-0.523239


In [27]:
senbet_port = portfolio.to_numpy()
senbet_avg_returns = compute_average(senbet_port)
print(senbet_avg_returns)

[0.05954910350779143, nan, -0.12332540201835579, -0.018038642190401744, nan, nan, -0.0025760755152972425, -0.0032278712994434286, nan, 0.06468577029135993, nan, nan, nan, nan, 0.09948168451829906, -0.07047934045865906, 0.02065302659457953, 0.022234293187041898, nan, nan, -0.015607613093431243, nan, nan, 0.11249336487167837, nan, nan, nan, 0.08334750152718164, nan, 0.03869417229456235, nan, -0.048508254878328, nan, -0.03828654843927894, nan, nan, nan, nan, nan, -0.0015459307577350468, nan, nan, 0.048288934885530654, nan, nan, -0.41200784165355364]


In [28]:
senbet_demeaned = compute_stocks_demeaned(senbet_port, senbet_avg_returns)
# print(senbet_demeaned)

In [29]:
senbet_covmat = compute_covariance_matrix(senbet_demeaned)
print(senbet_covmat)

NameError: name 'np_demeaned' is not defined

In [ ]:
senbet_asset_weights = list(trimmed_senbet['Allocation']/100)

In [ ]:
est_risk = estimated_portfolio_risk_based_on_stddev(senbet_asset_weights, senbet_covmat)
print(est_risk)

[[16.75346446]]


In [ ]:
exp_ret = expected_portfolio_return(asset_weights=senbet_asset_weights, average_closing_prices = senbet_avg_returns)
print(exp_ret)

280.82169842356836


In [ ]:
sharpe_ratio(exp_ret,est_risk, 6)

matrix([[16.40387271]])